## Shapley 值的计算

### 注意事项

0. **这是本实验应该看的第一个文件，请根据本文件的指示，逐步完成实验**
1. 本实验需要填空的文件夹为 exactSV.py 和 mcSV.py，**请不要修改其他文件，包括本 ipynb 文件**
2. 完成两个文件函数代码的实现后，在以下每个代码框中验证计算结果
3. 在实验过程中，**允许使用生成式人工智能理解代码框架并完成实验**
4. **提交时需要将整个代码文件夹压缩成 .zip 格式提交，并且注意保留本 ipynb 文件的所有运行结果**，不要清除输出
5. （**很重要**）建议每次更新代码后，在运行 ipynb 文件前先重启（restart），否则其中一些记忆变量会干扰实验结果

文件内容解释：

- dataset: iris 训练与测试数据
- groundTruth: 不同任务 Shapley 值计算的正确答案
- exactSV.py：代补充 Shapley 值准确算法（基于联盟）的代码文件；
- game.py: 定义了几个 Shapley 值计算任务，具体内容无需理解，可以视为黑箱；
- homework.ipynb: 运行测试的文件；
- mcSV: 待补充采样算法近似计算 Shapley 值算法的代码文件；
- naiveSV: 已给出的 Shapley 值准确算法（基于排列）的代码问题；
- util.py: 定义了一些函数，具体内容无需理解，可以视为黑箱。

### 任务一：理解 naiveSV.py 中的 Shapley 值计算代码

naiveSV.py 中给出了使用公式
$$ SV_i = \frac{1}{n!} \sum_{\sigma \in S_n} (u(P_i(\sigma) \cup \{i\}) - u(P_i(\sigma))) $$
准确计算 Shapley 值的算法。要求同学们根据给出的注释理解代码，在接下来的填空中可以仿照这一代码完成实验。

### 任务二：补全 exactSV.py 中的 Shapley 值计算代码，并运行接下来的两个代码框验证结果

exactSV.py 中希望你补全基于公式
$$ SV_i = \frac{1}{n!} \sum_{S \subseteq N \setminus \{i\}} |S|!(n - |S| - 1)! (u(S \cup \{i\}) - u(S)) $$
准确计算 Shapley 值的算法。要求同学们根据注释补充完成相应的代码，然后运行接下来的两个代码框验证结果准确性。

In [ ]:
import os
import numpy as np
from exactSV import exact_sv
from game import Game

game_type = 'airport'
num_players = 10
game = Game(gt = game_type, n = num_players)
mySV = exact_sv(game) # 你计算的 Shapley 值
print("My Results:", mySV)
gt_path = f"./groundTruth/{game_type}_{num_players}.txt"

# 判断你的答案是否正确（即与标准答案相差有限）
if os.path.exists(gt_path):
    try:
        gt_values = np.loadtxt(gt_path) # Shapley 值标准答案
        print("Ground Truth:", gt_values)
        error = np.abs(mySV - gt_values)
        threshold = 1e-2
        is_correct = np.abs(error) < threshold
        accuracy = np.mean(is_correct) * 100
        print(f"Accuracy: {accuracy:.2f}%")
    except Exception as e:
        print(f"Accuracy caculation wrong: {e}")
else:
    print(f"Ground truth not exists: {gt_path}")

Number of player:  10
Generated 1023 Sets


100%|██████████| 1023/1023 [00:00<00:00, 150442.59it/s]

My Results: [0.1        0.21111111 0.33611111 0.47896825 0.64563492 0.84563492
 1.09563492 1.42896825 1.92896825 2.92896825]
Ground Truth: [0.1        0.21111111 0.33611111 0.47896825 0.64563492 0.84563492
 1.09563492 1.42896825 1.92896825 2.92896825]
Accuracy: 100.00%


检查扩展性：当 $n = 20$ 时运算可能偏慢，该部分检查你的算法是否能在合理的时间内计算完成。

In [3]:
import os
import numpy as np
from exactSV import exact_sv
from game import Game

game_type = 'airport'
num_players = 20
game = Game(gt = game_type, n = num_players)
mySV = exact_sv(game)
print("My Results:", mySV)
gt_path = f"./groundTruth/{game_type}_{num_players}.txt"
if os.path.exists(gt_path):
    try:
        gt_values = np.loadtxt(gt_path)
        print("Ground Truth:", gt_values)
        error = np.abs(mySV - gt_values)
        threshold = 1e-2
        is_correct = np.abs(error) < threshold
        accuracy = np.mean(is_correct) * 100
        print(f"Accuracy: {accuracy:.2f}%")
    except Exception as e:
        print(f"Accuracy caculation wrong: {e}")
else:
    print(f"Ground truth not exists: {gt_path}")

Number of player:  20
Generated 1048575 Sets


100%|██████████| 1048575/1048575 [00:09<00:00, 109005.61it/s]


My Results: [0.05       0.10263158 0.15818713 0.21701066 0.27951066 0.34617733
 0.4176059  0.49452898 0.57786231 0.6687714  0.7687714  0.87988251
 1.00488251 1.14773966 1.31440632 1.51440632 1.76440632 2.09773966
 2.59773966 3.59773966]
Ground Truth: [0.05       0.10263158 0.15818713 0.21701066 0.27951066 0.34617733
 0.4176059  0.49452898 0.57786231 0.6687714  0.7687714  0.87988251
 1.00488251 1.14773966 1.31440632 1.51440632 1.76440632 2.09773966
 2.59773966 3.59773966]
Accuracy: 100.00%


### 任务三：补全 mcSV.py 中的 Shapley 值采样算法，并运行下面的代码框验证结果

使用蒙特卡洛方法计算给定博弈中每个玩家的 Shapley 值。蒙特卡洛方法通过随机采样玩家排列来估算 Shapley 值，适用于玩家数量较多时的近似计算。

实现非常简单：回忆基于排列的准确 Shapley 值计算算法，需要计算 n! 个排列中所有玩家的边际贡献，蒙特卡洛采样算法从 n! 个排列中随机出一部分计算所有玩家的边际贡献作为 Shapley 值的近似解。

实现完成后运行下面的代码检查采样方法实现准确性，注意**可以调整采样次数，但不要修改误差标准**。

In [ ]:
import os
import numpy as np
from mcSV import mc_sv
from game import Game

threshold = 1e-2
game_type = 'airport'
num_players = 25
total_sample_budgets = 1600000 * num_players
game = Game(gt = game_type, n = num_players)
mySV = mc_sv(game, total_sample_budget = total_sample_budgets)
print("My Results:", mySV)
gt_path = f"./groundTruth/{game_type}_{num_players}.txt"
if os.path.exists(gt_path):
    try:
        gt_values = np.loadtxt(gt_path)
        print("Ground Truth:", gt_values)
        error = np.abs(mySV - gt_values)
        is_correct = np.abs(error) < threshold
        accuracy = np.mean(is_correct) * 100
        print(f"Accuracy: {accuracy:.2f}%")
    except Exception as e:
        print(f"Accuracy caculation wrong: {e}")
else:
    print(f"Ground truth not exists: {gt_path}")

检查其他任务的采样准确性：
1. voting：51人投票计算每个人的权利大小；
2. iris 机器学习分类模型，注意使用已经给定的 util 中 get dataset 的方式划分数据集合（通俗而言就是不要动代码），否则会导致 Shapley 值答案变化。

In [ ]:
## voting
import os
import numpy as np
from mcSV import mc_sv
from game import Game

threshold = 1e-2
game_type = 'voting'
num_players = 51
total_sample_budgets = 800000 *num_players
game = Game(gt = game_type, n = num_players)
mySV = mc_sv(game, total_sample_budget = total_sample_budgets)
print("My Results:", mySV)
gt_path = f"./groundTruth/{game_type}_{num_players}.txt"
if os.path.exists(gt_path):
    try:
        gt_values = np.loadtxt(gt_path)
        print("Ground Truth:", gt_values)
        error = np.abs(mySV - gt_values)
        is_correct = np.abs(error) < threshold
        accuracy = np.mean(is_correct) * 100
        print(f"Accuracy: {accuracy:.2f}%")
    except Exception as e:
        print(f"Accuracy caculation wrong: {e}")
else:
    print(f"Ground truth not exists: {gt_path}")

In [ ]:
## iris classfication
import os
import numpy as np
from sklearn import svm
from mcSV import mc_sv
from game import Game
from util import get_dataset

threshold = 1e-2
game_type = 'iris'
num_players = 100
total_sample_budgets = 10000 * num_players
datapath = './dataset'
trnX, trnY, tstX, tstY = get_dataset(game_type, datapath, trainsize = 150 - num_players)
mymodel = svm.SVC(decision_function_shape='ovo', probability=True)
game = Game(gt = game_type, x_train = trnX, y_train = trnY, x_test = tstX, y_test = tstY,
            model = mymodel, n = num_players)
mySV = mc_sv(game, total_sample_budget = total_sample_budgets)
print("My Results:", mySV)
gt_path = f"./groundTruth/{game_type}_{num_players}.txt"
if os.path.exists(gt_path):
    try:
        gt_values = np.loadtxt(gt_path)
        print("Ground Truth:", gt_values)
        error = np.abs(mySV - gt_values)
        is_correct = np.abs(error) < threshold
        accuracy = np.mean(is_correct) * 100
        print(f"Accuracy: {accuracy:.2f}%")
    except Exception as e:
        print(f"Accuracy caculation wrong: {e}")
else:
    print(f"Ground truth not exists: {gt_path}")

测试采样算法的收敛情况，即随着采样次数的增大，采样得到的 Shapley 近似值与真实值之间的均方误差降低的情况。下面的代码通过绘制均方误差随采样次数变化的曲线应当能展示这一点。

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mcSV import mc_sv
from game import Game

threshold = 1e-2
game_type = 'voting'
num_players = 51
min_total_sample_budgets = num_players * 2
max_total_sample_budgets = num_players * 800000
game = Game(gt = game_type, n = num_players)
gt_path = f"./groundTruth/{game_type}_{num_players}.txt"
if os.path.exists(gt_path):
    try:
        gt_values = np.loadtxt(gt_path)
        print("Ground Truth:", gt_values)
    except Exception as e:
        print(f"Error loading ground truth: {e}")
        gt_values = None
else:
    print(f"Ground truth not exists: {gt_path}")

if gt_values is None:
    print("Cannot proceed with MSE calculation without ground truth. Exiting.")
    exit()
else:
    #######examples#################
    sample_budgets_to_test = np.logspace(np.log10(min_total_sample_budgets), 
                                         np.log10(max_total_sample_budgets), num=10, dtype=int)
    sample_budgets_to_test = np.maximum(num_players, sample_budgets_to_test)
    sample_budgets_to_test = np.unique(sample_budgets_to_test)
    print(f"Testing with sample budgets: {sample_budgets_to_test}")
    ################################

    mse_values = []

    for budget in sample_budgets_to_test:
        print(f"\nCalculating Shapley Values for budget = {budget}...")
        current_mySV = mc_sv(game, total_sample_budget=budget)
        
        if current_mySV is None or len(current_mySV) != len(gt_values):
            print(f"Error: mc_sv did not return valid Shapley values for budget {budget}. Expected length {len(gt_values)}, got {len(current_mySV) if current_mySV is not None else 'None'}.")
            mse_values.append(np.nan)
            continue

        mse = np.mean((current_mySV - gt_values)**2)
        mse_values.append(mse)
        print(f"Budget: {budget}, MSE: {mse:.6e}")

    plt.figure(figsize=(10, 6))
    plt.plot(sample_budgets_to_test, mse_values, marker='o', linestyle='-')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel("Total Sample Budget")
    plt.ylabel("MSE")
    plt.title(f"'{game_type}'game's MSE vs Shapley sampling times (N={num_players})")
    plt.grid(True, which="both", ls="-")
    plt.tight_layout()
    plt.show()